<a href="https://colab.research.google.com/github/mratanusarkar/Web-Scraping-tickertapeIN/blob/feature%2Ffilter-and-sort/Notebooks/filter_and_sort_tickertapeIN_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Filtering and Sorting data scraped from tickertape.in

Input: 
* scraped JSON data (ETFs or Stocks)
* list of keys and values and comparison type for filtering
* list of keys and the order of the values for sorting

Output: JSON data with the filters & sortings applied on the input data

## Import Packages

In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

import json
import time
import datetime as dt

import matplotlib.pyplot as plt
import plotly.express as px

## Input JSON Data from Scraping Scripts/Notebooks

In [ ]:
with open('all-stocks-tickertape-data.json', 'r') as fp:
    stocksData = json.load(fp)

stocksData[0]

In [ ]:
with open('all-etfs-tickertape-data.json', 'r') as fp:
    etfsData = json.load(fp)

etfsData[0]

## Input Filter/Sort Parameters

In [ ]:
# TODO

## Data Analysis and EDA

In [ ]:
# TODO

## Filter and Sort Data based on Input Parameters

In [ ]:
# TODO